## AWS Glue DQ

This notebook creates a table in Redshift, along with an AWS Glue ETL job that includes data quality checks and anomaly detection.

In [ ]:
# 1. Create a python environment

# !conda create -y --name text-to-sql python=3.11.8
# !conda init && activate text-to-sql
# !conda install -n text-to-sql ipykernel --update-deps --force-reinstall -y
# !conda install -c conda-forge ipython-sql

## OR
# !python3 -m venv venv
# !source venv/bin/activate  # On Windows, use `venv\Scripts\activate`

# install ipykernel, which consists of IPython as well
# !pip install ipykernel
# create a kernel that can be used to run notebook commands inside the virtual environment
# !python3 -m ipykernel install --user --name=venv

In [ ]:
# 2. Install dependencies

# !pip install -r requirements.txt

### Set up environment variables

In [1]:
# 3. Import necessary libraries and load environment variables

from dotenv import load_dotenv, find_dotenv
import os
import boto3

# loading environment variables that are stored in local file
local_env_filename = 'dev.env'
load_dotenv(find_dotenv(local_env_filename),override=True)

os.environ['REGION'] = os.getenv('REGION')
os.environ['SQL_DATABASE'] = os.getenv('SQL_DATABASE') # LOCAL, SQLALCHEMY, REDSHIFT
os.environ['SQL_DIALECT'] = os.getenv('SQL_DIALECT') # SQlite, PostgreSQL
os.environ['DATABASE_SECRET_NAME'] = os.getenv('DATABASE_SECRET_NAME')
os.environ['S3_BUCKET_NAME'] = os.getenv('S3_BUCKET_NAME')
os.environ['GLUE_IAM_ROLE_ARN'] = os.getenv('GLUE_IAM_ROLE_ARN')


REGION = os.environ['REGION']
SQL_DATABASE = os.environ['SQL_DATABASE']
SQL_DIALECT = os.environ['SQL_DIALECT']
DATABASE_SECRET_NAME = os.environ['DATABASE_SECRET_NAME']
S3_BUCKET_NAME = os.environ['S3_BUCKET_NAME']
GLUE_IAM_ROLE_ARN = os.environ['GLUE_IAM_ROLE_ARN']
print(f"Using database: {SQL_DATABASE} with sql dialect: {SQL_DIALECT} in region: {REGION}")

Using database: REDSHIFT with sql dialect: PostgreSQL in region: us-east-1


In [2]:
# Initialize DatabaseUtil for Redshift

from utils.database import DatabaseUtil

db_util = DatabaseUtil(
                sql_database= SQL_DATABASE,
                region=REGION,
                secret_name=DATABASE_SECRET_NAME
)

In [16]:
# Create table in Redshift (if table does not exist)

sql_statements = ['''CREATE TABLE public.syn_data
(
    id integer,
    timestamp bigint,
    name character varying(20),
    version integer DEFAULT 1
)
DISTSTYLE EVEN;''']
db_util.create_database_tables(sql_statements)

SQL execution completed.


In [4]:
# upload all etl csv files to S3 bucket
s3 = boto3.client('s3')

prefix = 'etl-data'


# create prefix 'transforms' in S3 bucket
s3.put_object(Bucket=S3_BUCKET_NAME, Key=f'{prefix}/')

# upload all files from glue-component to s3_path
for file in os.listdir('syn-data'):
    try:
        s3.upload_file(
            f'syn-data/{file}',  # Local file path
            S3_BUCKET_NAME,               # S3 bucket name
            f'{prefix}/{file}'         # S3 key (path in bucket)
        )
        print(f"Successfully uploaded CSV file to s3://{S3_BUCKET_NAME}/{prefix}/{file}")

    except Exception as e:
        print(f"Error uploading CSV file to S3: {e}")

Successfully uploaded CSV file to s3://felixh-demo/etl-data/data_anomaly1.csv
Successfully uploaded CSV file to s3://felixh-demo/etl-data/data_anomaly0.csv
Successfully uploaded CSV file to s3://felixh-demo/etl-data/data_anomaly2.csv
Successfully uploaded CSV file to s3://felixh-demo/etl-data/data_anomaly3.csv


In [5]:
# test Redshift connection and query table
sql_statement = 'SELECT COUNT(*) FROM public.syn_data'
result = db_util.run_sql(sql_statement)
print(result)

  count
0  None


In [28]:
# create AWS Glue Redshift ETL job script

glue_job_script = f'''import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.functions import col, when, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType, ShortType
import logging
import boto3

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize the Glue context
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Get job parameters
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'input_path', 'output_path', 'redshift_connection', 'redshift_table'])
input_path = args['input_path']
output_path = args['output_path']
redshift_connection = args['redshift_connection']
redshift_table = args['redshift_table']

logger.info(f'Input path: {{input_path}}')
logger.info(f'Redshift connection: {{redshift_connection}}')
logger.info(f'Redshift table: {{redshift_table}}')


# Set up the job
job.init(args['JOB_NAME'], args)

def get_redshift_connection_details(connection_name):
    glue_client = boto3.client('glue')
    try:
        response = glue_client.get_connection(Name=connection_name)
        connection_properties = response['Connection']['ConnectionProperties']
        
        return {{
            'jdbc_url': connection_properties['JDBC_CONNECTION_URL'],
            'username': connection_properties['USERNAME'],
            'password': connection_properties['PASSWORD']
        }}
    except Exception as e:
        logger.error(f"Error getting Redshift connection details: {{e}}")
        raise

def get_redshift_table_schema(connection_details, table_name):
    try:
        # Use Spark to query Redshift
        df = spark.read.format("jdbc").option("url", connection_details['jdbc_url']).option("dbtable", f"information_schema.columns").option("user", connection_details['username']).option("password", connection_details['password']).option("driver", "com.amazon.redshift.jdbc42.Driver").load()
        
        # Filter for the specific table and get column information
        columns_info = df.filter((df.table_schema == table_name.split('.')[0]) & 
                                 (df.table_name == table_name.split('.')[1])).select("column_name", "data_type").collect()
        
        # Create a schema based on Redshift data types
        schema = StructType()
        for col_info in columns_info:
            col_name = col_info['column_name']
            data_type = col_info['data_type']
            if data_type == 'integer':
                schema.add(StructField(col_name, IntegerType()))
            elif data_type == 'bigint':
                schema.add(StructField(col_name, LongType()))
            elif data_type in ['double precision', 'real']:
                schema.add(StructField(col_name, DoubleType()))
            elif data_type == 'smallint':
                schema.add(StructField(col_name, ShortType()))
            else:
                schema.add(StructField(col_name, StringType()))
        
        return schema
    except Exception as e:
        logger.error(f"Error getting Redshift table schema: {{e}}")
        raise

try:
    # Get Redshift connection details
    connection_details = get_redshift_connection_details(redshift_connection)
    
    # Get Redshift table schema
    redshift_schema = get_redshift_table_schema(connection_details, redshift_table)
    logger.info(f"Redshift table schema: {{redshift_schema}}")

    # Read the CSV file from S3
    logger.info(f"Reading data from {{input_path}}")
    if not input_path.startswith('s3://'):
        raise ValueError(f"Invalid S3 path: {{input_path}}. Path must start with 's3://'")
    df = spark.read.csv(input_path, header=True, inferSchema=True)
    
    logger.info(f"Read {{df.count()}} records from S3")

    # AWS GLUE DQ CHECKS
    # https://docs.aws.amazon.com/glue/latest/dg/data-quality-gs-studio-notebooks.html
    from awsgluedq.transforms import EvaluateDataQuality

    EvaluateDataQuality_ruleset = """
        Rules = [
            ColumnDataType "id" = "Integer",
            ColumnDataType "version" = "Integer",
            ColumnLength "name" <= 20
            
        ]
        Analyzers = [
        RowCount ,
        ColumnCount ,
        ColumnLength "name",
        Completeness "id",
        Completeness "version"
        ]
    """

    # convert dataframe to dynamic frame
    dyf = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")

    EvaluateDataQualityMultiframe = EvaluateDataQuality().process_rows(
    frame=dyf,
    ruleset=EvaluateDataQuality_ruleset,
    publishing_options={{
            "dataQualityEvaluationContext": "EvaluateDataQualityMultiframe",
            "enableDataQualityCloudWatchMetrics": False,
            "enableDataQualityResultsPublishing": False
        }},
        additional_options={{"performanceTuning.caching": "CACHE_NOTHING"}}
    )

    # review results
    ruleOutcomes = SelectFromCollection.apply(
        dfc=EvaluateDataQualityMultiframe,
        key="ruleOutcomes",
        transformation_ctx="ruleOutcomes",
    )

    ruleOutcomes.toDF().show(truncate=False)

    # review row level results
    rowLevelOutcomes = SelectFromCollection.apply(
        dfc=EvaluateDataQualityMultiframe,
        key="rowLevelOutcomes",
        transformation_ctx="rowLevelOutcomes",
    )

    rowLevelOutcomes_df = rowLevelOutcomes.toDF() # Convert Glue DynamicFrame to SparkSQL DataFrame
    rowLevelOutcomes_df_passed = rowLevelOutcomes_df.filter(rowLevelOutcomes_df.DataQualityEvaluationResult == "Passed") # Filter only the Passed records.
    rowLevelOutcomes_df.filter(rowLevelOutcomes_df.DataQualityEvaluationResult == "Failed").show(5, truncate=False) # Review the Failed records                    
    rowLevelOutcomes_df_error = rowLevelOutcomes_df.filter(rowLevelOutcomes_df.DataQualityEvaluationResult == "Failed")

    # write the Passed records to the destination. 
    # convert back to DynamicFrame before writing
    rowLevelOutcomes_dyf_passed = DynamicFrame.fromDF(rowLevelOutcomes_df_passed, glueContext, "passed_records")
    rowLevelOutcomes_dyf_error = DynamicFrame.fromDF(rowLevelOutcomes_df_error, glueContext, "error_records")
    
    # write error records to S3 destination for review
    glueContext.write_dynamic_frame.from_options(
        frame = rowLevelOutcomes_dyf_error,
        connection_type = "s3",
        connection_options = {{"path": f'{{output_path}}/etl_detected_dq_errors'}},
        format = "json")
         
    # write records that passed Glue DQ checks to Amazon Redshift
    original_columns = df.columns  # These are the columns from your input data
    rowLevelOutcomes_df_passed_flat = rowLevelOutcomes_df_passed.select(original_columns)

    df = rowLevelOutcomes_df_passed_flat

    # align df schema with Redshift schema
    for field in redshift_schema.fields:
        if field.name in df.columns:
            if isinstance(field.dataType, (IntegerType, LongType, ShortType)):
                df = df.withColumn(field.name, col(field.name).cast(field.dataType))
            elif isinstance(field.dataType, DoubleType):
                df = df.withColumn(field.name, col(field.name).cast(DoubleType()))
            else:
                df = df.withColumn(field.name, col(field.name).cast(StringType()))
        else:
            df = df.withColumn(field.name, lit(None).cast(field.dataType))

    # handle null values (optional, adjust as needed)
    for column in df.columns:
        df = df.withColumn(column, when(col(column) == "", None).otherwise(col(column)))
    
    # Convert back to DynamicFrame with only the original columns
    dynamic_frame = DynamicFrame.fromDF(df, glueContext, "flattened_passed_records")

    # Write to Redshift
    logger.info(f"Writing data to Redshift table {{redshift_table}}")
    glueContext.write_dynamic_frame.from_jdbc_conf(
        frame=dynamic_frame,
        catalog_connection=redshift_connection,
        connection_options={{
            "dbtable": redshift_table,
            "database": connection_details['jdbc_url'].split('/')[-1]
        }},
        redshift_tmp_dir=f"s3://{S3_BUCKET_NAME}/redshift-tmp/",
        transformation_ctx="datasink"
    )
    
    logger.info("Data successfully written to Redshift")

except Exception as e:
    logger.error(f"An error occurred: {{str(e)}}")
    raise

# Commit the job
job.commit()
'''

# Upload the updated script to S3
s3_client = boto3.client('s3')
object_key = 'glue/glue-etl-processing.py'

try:
    s3_client.put_object(Bucket=S3_BUCKET_NAME, Key=object_key, Body=glue_job_script)
    print(f"Successfully uploaded updated Glue job script to s3://{S3_BUCKET_NAME}/{object_key}")
except Exception as e:
    print(f"Error uploading Glue job script to S3: {e}")

s3_script_path = f's3://{S3_BUCKET_NAME}/{object_key}'
print(f"S3 script path: {s3_script_path}")

Successfully uploaded updated Glue job script to s3://felixh-demo/glue/glue-etl-processing.py
S3 script path: s3://felixh-demo/glue/glue-etl-processing.py


In [7]:
glue_client = boto3.client('glue', region_name=REGION)

glue_job_name = 'RedshiftETL-DQ'

In [ ]:
# create AWS Glue job via boto3
 
response = glue_client.create_job(
    Name=glue_job_name,
    Description='ETL job to load data from S3 to Redshift',
    Role=GLUE_IAM_ROLE_ARN,
    ExecutionProperty={
        'MaxConcurrentRuns': 1
    },
    DefaultArguments={
        '--connection-names': 'dev-redshift-connection'
    },
    Command={
        'Name': 'glueetl',
        'ScriptLocation': s3_script_path,
        'PythonVersion': '3'
    },
    MaxRetries=2,
    Timeout=1440,
    Tags={
        'usecase': 'Glue ETL DQ'
    },
    GlueVersion='4.0',
    NumberOfWorkers=1,
    WorkerType='Standard',
    Connections={
        'Connections': ['dev-redshift-connection']
    }
)
print(response)

In [15]:
# run AWS Glue job with arguments

args = {
    '--JOB_NAME': glue_job_name,
    '--input_path': f's3://felixh-demo/etl-data/data_anomaly0.csv',
    '--output_path': f's3://felixh-demo/',
    '--redshift_connection': 'dev-redshift-connection',
    '--redshift_table': 'public.syn_data'
}

response = glue_client.start_job_run(
    JobName=glue_job_name,
    Arguments=args
)
print(response)

{'JobRunId': 'jr_d6ae499508d19f36e2b7b1bf05da785db70140d5e57aa161694859ef4d030e26', 'ResponseMetadata': {'RequestId': '66b1f961-ae04-4d32-802d-3400a67c971a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 13 Nov 2024 20:01:07 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '82', 'connection': 'keep-alive', 'x-amzn-requestid': '66b1f961-ae04-4d32-802d-3400a67c971a'}, 'RetryAttempts': 0}}


In [29]:
# run AWS Glue job again with different input file that has anomalies
args = {
    '--JOB_NAME': glue_job_name,
    '--input_path': f's3://felixh-demo/etl-data/data_anomaly2.csv',
    '--output_path': f's3://felixh-demo/',
    '--redshift_connection': 'dev-redshift-connection',
    '--redshift_table': 'public.syn_data'
}

response = glue_client.start_job_run(
    JobName=glue_job_name,
    Arguments=args
)
job_run_id = response['JobRunId']
print(f'Job run ID: {job_run_id}')

Job run ID: jr_d44779cd138f335bc8d851a1651757e08fc9086527072dc1ebaf0c49b6acbc32


In [30]:
# wait on the status of the Glue job

import time

def wait_for_job_completion(job_name, run_id, max_attempts=30):
    """Wait for an AWS Glue job to complete, checking status every 30 seconds."""
    for i in range(max_attempts):
        response = glue_client.get_job_run(JobName=job_name, RunId=run_id)
        status = response['JobRun']['JobRunState']
        
        print(f"Job status: {status} (attempt {i+1}/{max_attempts})")
        
        if status in ['SUCCEEDED', 'FAILED', 'STOPPED', 'TIMEOUT']:
            return status
            
        time.sleep(30)
    
    return 'TIMEOUT'

# Wait for job completion
final_status = wait_for_job_completion(glue_job_name, job_run_id)
print(f"Final job status: {final_status}")

Job status: WAITING (attempt 1/30)
Job status: RUNNING (attempt 2/30)
Job status: RUNNING (attempt 3/30)
Job status: RUNNING (attempt 4/30)
Job status: RUNNING (attempt 5/30)
Job status: RUNNING (attempt 6/30)
Job status: SUCCEEDED (attempt 7/30)
Final job status: SUCCEEDED


In [31]:
# this last job should not have loaded any records to Redshift
# because all of the rows had data quality issues

# so let'scheck if any records were loaded to Redshift
sql_statement = 'SELECT * FROM public.syn_data'
result = db_util.run_sql(sql_statement)
print(result)

Empty DataFrame
Columns: [id, timestamp, name, version]
Index: []


## Conclusion
This notebook demonstrates how to use AWS Glue Data Quality to check for data quality issues in a CSV file and then load qualified records into an Amazon Redshift table.